In [190]:
import pandas as pd
import os
import numpy as np

from tbparse import SummaryReader

In [191]:
os.getcwd()
# change directory to assembly_glovebox_dataet 
os.chdir('/home/shivansh/repos/assembly_glovebox_dataset')

In [192]:
versions = {
    'unet': {
        'ood': 'version_153',
        'ood, dropout': 'version_154',
        'id': 'version_155',
        'id, dropout': 'version_156',
        'replaced_gs, ood': 'version_161',
        'replaced_gs, ood, dropout': 'version_162',
        'replaced_gs': 'version_163',
        'replaced_gs, dropout': 'version_164'
    },
    'bisenet': {
        'ood': 'version_149',
        'ood, dropout': 'version_150',
        'id': 'version_151',
        'id, dropout': 'version_152',
        'replaced_gs, ood': 'version_165',
        'replaced_gs, ood, dropout': 'version_166',
        'replaced_gs': 'version_167',
        'replaced_gs, dropout': 'version_168'
    },
    'mobilesam': {
        'ood': 'version_157',
        'ood, dropout': 'version_158',
        'id': 'version_159',
        'id, dropout': 'version_160',
        'replaced_gs, ood': 'version_170',
        'replaced_gs, ood, dropout': 'version_171',
        'replaced_gs': 'version_172',
        'replaced_gs, dropout': 'version_173'
    }
}

In [193]:
# create pandas dataframe

dfs = []

for model in versions:
    for set in versions[model]:
        version = versions[model][set]
        log_dir = f"./lightning_logs/{version}"
        reader =  SummaryReader(log_dir)
        df = reader.scalars

        # TODO: rename respective versions to type with dictionary
        df['type'] = set
        df['model'] = model

        dfs.append(df[~df['tag'].isin(['epoch', 'test_loss'])])

result = pd.concat(dfs)

In [194]:
result

,step,tag,value,type,model
0,0,avg_frame_reference_time,0.004106,ood,unet
2,0,test_ace,10.057115,ood,unet
3,0,test_iou,0.570464,ood,unet
0,0,avg_frame_reference_time,0.004627,"ood, dropout",unet
2,0,test_ace,10.047482,"ood, dropout",unet
...,...,...,...,...,...
2,0,test_ace,8.739035,replaced_gs,mobilesam
3,0,test_iou,0.516547,replaced_gs,mobilesam
0,0,avg_frame_reference_time,0.000000,"replaced_gs, dropout",mobilesam
2,0,test_ace,8.723783,"replaced_gs, dropout",mobilesam


In [195]:
dfs = {value: result.loc[result['model'] == value] for value in ['bisenet', 'mobilesam', 'unet']}

In [196]:
y = dfs['mobilesam'].groupby(["tag"], group_keys=True).apply(lambda x: x.drop(columns=['step', 'tag']))
x = dfs['bisenet'].groupby(["tag"], group_keys=True).apply(lambda x: x.drop(columns=['step', 'tag']))

In [197]:
dfs['unet']

,step,tag,value,type,model
0,0,avg_frame_reference_time,0.004106,ood,unet
2,0,test_ace,10.057115,ood,unet
3,0,test_iou,0.570464,ood,unet
0,0,avg_frame_reference_time,0.004627,"ood, dropout",unet
2,0,test_ace,10.047482,"ood, dropout",unet
3,0,test_iou,0.573503,"ood, dropout",unet
0,0,avg_frame_reference_time,0.007269,id,unet
2,0,test_ace,9.980969,id,unet
3,0,test_iou,0.799321,id,unet
0,0,avg_frame_reference_time,0.007681,"id, dropout",unet


In [198]:
x = dfs['mobilesam'].merge(dfs['bisenet'], on=['tag', 'type']).groupby(["tag"], group_keys=True).apply(lambda x: x.drop(columns=['step_x', 'tag', 'step_y']))
x = x.merge(dfs['unet'], on=['tag', 'type']).groupby(["tag"], group_keys=True).apply(lambda x: x.drop(columns=['tag', 'step']))


x = x.rename(columns={'value_x': 'mobilesam', 'value_y': 'bisenet', 'value': 'unet'})


x.drop(columns=['model_x', 'model_y', 'model'], inplace=True)
# columns_to_drop = ['model_x', 'model_y', 'model']
# df = df.drop(columns=columns_to_drop)

# TODO: remove epoch columns
# TODO: fix avg time reference values

In [199]:
x = x[['type', 'bisenet', 'mobilesam', 'unet']]

x

type    bisenet  mobilesam  \
tag                                                                            
avg_frame_reference_time 0                         ood   0.006823   0.000000   
                         1                ood, dropout   0.007532   0.000000   
                         2                          id   0.013415   0.000000   
                         3                 id, dropout   0.014159   0.000000   
                         4            replaced_gs, ood   0.004507   0.000000   
                         5   replaced_gs, ood, dropout   0.004763   0.000000   
                         6                 replaced_gs   0.010148   0.000000   
                         7        replaced_gs, dropout   0.011199   0.000000   
test_ace                 8                         ood  19.734829   8.781267   
                         9                ood, dropout  19.708696   8.760837   
                         10                         id   9.925502   6.449127   
                         11                id, dropout   9.928941   6.446730   
                         12           replaced_gs, ood  17.835138   9.997455   
                         13  replaced_gs, ood, dropout  17.823523   9.976060   
                         14                replaced_gs  13.221034   8.739035   
                         15       replaced_gs, dropout  13.219630   8.723783   
test_iou                 16                        ood   0.541172   0.522928   
                         17               ood, dropout   0.548948   0.507600   
                         18                         id   0.737037   0.560453   
                         19                id, dropout   0.728564   0.561671   
                         20           replaced_gs, ood   0.529273   0.521747   
                         21  replaced_gs, ood, dropout   0.539117   0.513460   
                         22                replaced_gs   0.540659   0.516547   
                         23       replaced_gs, dropout   0.537827   0.494581   

                                  unet  
tag                                     
avg_frame_reference_time 0    0.004106  
                         1    0.004627  
                         2    0.007269  
                         3    0.007681  
                         4    0.003354  
                         5    0.003556  
                         6    0.006386  
                         7    0.006523  
test_ace                 8   10.057115  
                         9   10.047482  
                         10   9.980969  
                         11   9.977846  
                         12  10.341810  
                         13  10.330724  
                         14   9.568053  
                         15   9.556040  
test_iou                 16   0.570464  
                         17   0.573503  
                         18   0.799321  
                         19   0.784424  
                         20   0.568530  
                         21   0.566153  
                         22   0.550574  
                         23   0.568284

In [200]:
result['bisenet'] = result.apply(lambda x: x['value'] if x['model'] == 'bisenet' else None, axis=1)
result['mobilesam'] = result.apply(lambda x: x['value'] if x['model'] == 'mobilesam' else None, axis=1)

In [201]:
result = result.drop(columns=['value', 'model'])

In [202]:
x = result.groupby(["tag", "type"], group_keys=True).apply(lambda x: x.drop(columns=['step', 'tag']))

In [203]:
x

type  \
tag                      type                                                     
avg_frame_reference_time id                        0                         id   
                                                   0                         id   
                                                   0                         id   
                         id, dropout               0                id, dropout   
                                                   0                id, dropout   
...                                                                         ...   
test_iou                 replaced_gs, ood          3           replaced_gs, ood   
                                                   3           replaced_gs, ood   
                         replaced_gs, ood, dropout 3  replaced_gs, ood, dropout   
                                                   3  replaced_gs, ood, dropout   
                                                   3  replaced_gs, ood, dropout   

                                                       bisenet  mobilesam  
tag                      type                                              
avg_frame_reference_time id                        0       NaN        NaN  
                                                   0  0.013415        NaN  
                                                   0       NaN   0.000000  
                         id, dropout               0       NaN        NaN  
                                                   0  0.014159        NaN  
...                                                        ...        ...  
test_iou                 replaced_gs, ood          3  0.529273        NaN  
                                                   3       NaN   0.521747  
                         replaced_gs, ood, dropout 3       NaN        NaN  
                                                   3  0.539117        NaN  
                                                   3       NaN   0.513460  

[72 rows x 3 columns]

In [204]:
x.T

tag       avg_frame_reference_time                                           \
type                            id                 id, dropout                
                                 0         0    0            0            0   
type                            id        id   id  id, dropout  id, dropout   
bisenet                        NaN  0.013415  NaN          NaN     0.014159   
mobilesam                      NaN       NaN  0.0          NaN          NaN   

tag                                                       ...     test_iou  \
type                    ood                 ood, dropout  ...  replaced_gs   
                     0    0         0    0             0  ...            3   
type       id, dropout  ood       ood  ood  ood, dropout  ...  replaced_gs   
bisenet            NaN  NaN  0.006823  NaN           NaN  ...          NaN   
mobilesam          0.0  NaN       NaN  0.0           NaN  ...     0.516547   

tag                                                                          \
type       replaced_gs, dropout                                               
                              3                     3                     3   
type       replaced_gs, dropout  replaced_gs, dropout  replaced_gs, dropout   
bisenet                     NaN              0.537827                   NaN   
mobilesam                   NaN                   NaN              0.494581   

tag                                                              \
type       replaced_gs, ood                                       
                          3                 3                 3   
type       replaced_gs, ood  replaced_gs, ood  replaced_gs, ood   
bisenet                 NaN          0.529273               NaN   
mobilesam               NaN               NaN          0.521747   

tag                                                              \
type       replaced_gs, ood, dropout                              
                                   3                          3   
type       replaced_gs, ood, dropout  replaced_gs, ood, dropout   
bisenet                          NaN                   0.539117   
mobilesam                        NaN                        NaN   

tag                                   
type                                  
                                   3  
type       replaced_gs, ood, dropout  
bisenet                          NaN  
mobilesam                    0.51346  

[3 rows x 72 columns]

In [205]:
# TODO: apply dictionary and automatically do this for all provided directories

# Reading Predictive Entropy Data

In [208]:
# writing out the mean predictive entropy just to get a good glance, box plots should be easy

# create pandas dataframe
import os
import glob


index = list(versions.keys())
columns = list(versions['unet'].keys())
df = pd.DataFrame(index = index, columns=columns)

# for model in versions:
#     for set in versions[model]:
#         version = versions[model][set]
#         csv = f"./lightning_logs/{version}/predictive_entropy.csv"

#         df = pd.read_csv(csv)

#         mean = df.mean()





#         # TODO: rename respective versions to type with dictionary
#         df['type'] = set
#         df['model'] = model

#         dfs.append(df[~df['tag'].isin(['epoch', 'test_loss'])])

# result = pd.concat(dfs)

In [209]:
df

,ood,"ood, dropout",id,"id, dropout","replaced_gs, ood","replaced_gs, ood, dropout",replaced_gs,"replaced_gs, dropout"
unet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bisenet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mobilesam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
versions.keys()

dict_keys(['unet', 'bisenet', 'mobilesam'])